In [ ]:
# !pip install pandas

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
%cd drive/MyDrive/chinese_text_classification/

/content/drive/MyDrive/chinese_text_classification


In [ ]:
import pandas as pd
df = pd.read_csv('./data.csv', encoding='gbk')
df.head()

,建言类型,问题,改进意见
0,安全隐患,用户隐私数据保护措施不完善,加强数据加密和权限申请，确保用户个人信息安全。
1,安全隐患,存在安全漏洞容易受到黑客攻击,加强网络安全防护，定期进行漏洞修复和安全测试，确保用户数据的安全性。
2,安全隐患,系统登录认证方式薄弱，易受到非法访问,改进系统登录认证机制，增加多重身份验证，提高账号安全性。
3,安全隐患,缺乏日志监控和异常检测机制,建立日志监控和异常检测系统，及时发现和应对安全威胁。
4,安全隐患,内部员工权限管理混乱，存在安全隐患,加强员工权限管理，实行权限分级制度，防止数据泄露。


In [ ]:
df.rename(columns={'建言类型':'label', '问题':'text', '改进意见':'suggestions'}, inplace=True)
# df.head()
df['label'] = df['label'].apply(lambda x: str(x))
df['text'] = df['text'].apply(lambda x: str(x))
df['suggestions'] = df['suggestions'].apply(lambda x: str(x))
df['text1'] = df['text'] + ',' + df['suggestions']

df.head()


,label,text,suggestions,text1
0,安全隐患,用户隐私数据保护措施不完善,加强数据加密和权限申请，确保用户个人信息安全。,"用户隐私数据保护措施不完善,加强数据加密和权限申请，确保用户个人信息安全。"
1,安全隐患,存在安全漏洞容易受到黑客攻击,加强网络安全防护，定期进行漏洞修复和安全测试，确保用户数据的安全性。,"存在安全漏洞容易受到黑客攻击,加强网络安全防护，定期进行漏洞修复和安全测试，确保用户数据的安全性。"
2,安全隐患,系统登录认证方式薄弱，易受到非法访问,改进系统登录认证机制，增加多重身份验证，提高账号安全性。,"系统登录认证方式薄弱，易受到非法访问,改进系统登录认证机制，增加多重身份验证，提高账号安全性。"
3,安全隐患,缺乏日志监控和异常检测机制,建立日志监控和异常检测系统，及时发现和应对安全威胁。,"缺乏日志监控和异常检测机制,建立日志监控和异常检测系统，及时发现和应对安全威胁。"
4,安全隐患,内部员工权限管理混乱，存在安全隐患,加强员工权限管理，实行权限分级制度，防止数据泄露。,"内部员工权限管理混乱，存在安全隐患,加强员工权限管理，实行权限分级制度，防止数据泄露。"


In [ ]:
temp = df['text1']
df.drop(labels=['text'], axis=1, inplace=True)
df.drop(labels=['suggestions'], axis=1, inplace=True)
df.rename(columns={'text1':'text'}, inplace=True)
df.head()

,label,text
0,安全隐患,"用户隐私数据保护措施不完善,加强数据加密和权限申请，确保用户个人信息安全。"
1,安全隐患,"存在安全漏洞容易受到黑客攻击,加强网络安全防护，定期进行漏洞修复和安全测试，确保用户数据的安全性。"
2,安全隐患,"系统登录认证方式薄弱，易受到非法访问,改进系统登录认证机制，增加多重身份验证，提高账号安全性。"
3,安全隐患,"缺乏日志监控和异常检测机制,建立日志监控和异常检测系统，及时发现和应对安全威胁。"
4,安全隐患,"内部员工权限管理混乱，存在安全隐患,加强员工权限管理，实行权限分级制度，防止数据泄露。"


### 将标签从汉字准换为数字

In [ ]:
df['label'] = df['label'].map({'安全隐患':0, '客户服务':1, '市场运营':2, '网络运营':3})

In [ ]:
# !pip install pip install scikit-learn
# df



In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(
    df['text'], df['label'], stratify=df['label']
)

# df['x_train'] = x_train
# df['y_train'] = y_train

x_train, x_val, y_train, y_val = train_test_split(
    x_train, y_train, stratify=y_train
)


In [ ]:
# y_train

## 预训练模型微调

In [ ]:
!pip install transformers

In [ ]:
from transformers import AutoModel, BertTokenizerFast
bert = AutoModel.from_pretrained('bert-base-chinese')
tokenizer = BertTokenizerFast.from_pretrained('bert-base-chinese')

In [ ]:
train_idx = x_train.dropna().index
val_idx = x_val.dropna().index
test_idx = x_test.dropna().index

train_tokens = tokenizer.batch_encode_plus(x_train[train_idx].to_list(),
                      max_length = 100,
                      pad_to_max_length = True,
                      truncation = True)
val_tokens = tokenizer.batch_encode_plus(x_val[val_idx].to_list(),
                      max_length = 100,
                      pad_to_max_length = True,
                      truncation = True)
test_tokens = tokenizer.batch_encode_plus(x_test[test_idx].to_list(),
                      max_length = 100,
                      pad_to_max_length = True,
                      truncation = True)


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2393: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [ ]:
import torch
import numpy as np
train_seq = torch.tensor(train_tokens['input_ids'])
train_mask = torch.tensor(train_tokens['attention_mask'])
train_y = torch.tensor(y_train[train_idx].to_list())

val_seq = torch.tensor(val_tokens['input_ids'])
val_mask = torch.tensor(val_tokens['attention_mask'])
val_y = torch.tensor(y_val[val_idx].to_list())

test_seq = torch.tensor(test_tokens['input_ids'])
test_mask = torch.tensor(test_tokens['attention_mask'])
test_y = torch.tensor(y_test[test_idx].to_list())

In [ ]:
from torch.utils.data import TensorDataset, RandomSampler, DataLoader
train_data = TensorDataset(train_seq, train_mask, train_y)
train_sampler = RandomSampler(train_data)
trainloader = DataLoader(train_data,
              sampler = train_sampler,
              batch_size = 32)

val_data = TensorDataset(val_seq, val_mask, val_y)
val_sampler = RandomSampler(val_data)
valloader = DataLoader(val_data,
              sampler = val_sampler,
              batch_size = 32)

test_data = TensorDataset(test_seq, test_mask, test_y)
test_sampler = RandomSampler(test_data)
testloader = DataLoader(test_data,
            sampler = test_sampler,
            batch_size = 32)

In [ ]:
# 冻结bert参数
for param in bert.parameters():
  param.requires_grad = False

from torch import nn
from transformers import AdamW
import torch.nn.functional as F
from sklearn.utils.class_weight import compute_class_weight


class BertClassifier(nn.Module):
    def __init__(self, bert):
        super().__init__()
        self.bert = bert
        self.fc1 = nn.Linear(768,4)

    def forward(self, sent_id, mask):
        _ , cls_hs = self.bert(sent_id, attention_mask = mask, return_dict=False)
        return F.log_softmax(self.fc1(cls_hs), dim = 1)

In [ ]:
model = BertClassifier(bert)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

optimizer = AdamW(model.parameters(), lr=1e-5)

class_weights = compute_class_weight(class_weight='balanced',
                    classes=np.unique(y_train),
                    y=y_train)

weights = torch.tensor(class_weights, dtype = torch.float)
weights = weights.to(device)

criterion = nn.NLLLoss(weight = weights)


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
! mkdir checkpoints

### 微调模型

In [ ]:
from tqdm.notebook import tqdm

epoches = 30
model_save = 1
save_path = './checkpoints/'
best_mean_loss = 1e5

for e in range(epoches):
    train_loss = 0.0
    for batch in tqdm(trainloader):
        batch = [i.to(device) for i in batch]
        sent_id, masks, labels = batch

        optimizer.zero_grad()
        preds = model(sent_id, masks)
        loss = criterion(preds, labels)
        train_loss += loss.item()

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()

    if e % model_save == 0:
      with torch.no_grad():
        val_loss = 0.0
        for batch in tqdm(valloader):
          batch = [i.to(device) for i in batch]
          sent_id, masks, labels = batch
          preds = model(sent_id, masks)
          loss = criterion(preds, labels)
          val_loss += loss.item()
          mean_loss = val_loss / len(trainloader)

      path = save_path + 'checkpoint_latest.pth'
      torch.save({
        'epoch': e+1,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss':val_loss
        }, path)

      if val_loss < best_mean_loss:
        best_mean_loss = val_loss
        path = save_path + 'checkpoint_best.pth'
        torch.save({
          'model_state_dict': model.state_dict(),
          'optimizer_state_dict': optimizer.state_dict()
        }, path)

    print(f'Epoch:{e+1}\t\tTraining Loss: {train_loss / len(trainloader)}')

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Epoch:1		Training Loss: 1.37037353515625


  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

...

  0%|          | 0/2 [00:00<?, ?it/s]

Epoch:29		Training Loss: 1.3558220863342285


  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Epoch:30		Training Loss: 1.359481930732727


In [ ]:
# 加载模型验证
from sklearn.metrics import confusion_matrix, classification_report
model = BertClassifier(bert)
model = model.to(device)
optimizer = AdamW(model.parameters(), lr=1e-5)

path = './checkpoints/checkpoint_best.pth'
checkpoint = torch.load(path)

model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
# epoch = checkpoint['epoch']

model.eval()

pred_label = []
true_label = []
for batch in tqdm(testloader):
    batch = [i.to(device) for i in batch]
    # batch
    sent_id, masks, labels = batch
    preds = model(sent_id, masks)
    pred_label.extend(torch.argmax(preds, axis = 1).to(device))
    true_label.extend(labels.to(device))


true_label = torch.tensor(true_label)
pred_label = torch.tensor(pred_label)

confusion_matrix(true_label, pred_label)
print(classification_report(true_label, pred_label))



/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/2 [00:00<?, ?it/s]

              precision    recall  f1-score   support

           0       0.46      0.27      0.34        22
           1       0.47      0.58      0.52        12
           2       0.83      0.33      0.48        15
           3       0.24      0.60      0.34        10

    accuracy                           0.41        59
   macro avg       0.50      0.45      0.42        59
weighted avg       0.52      0.41      0.41        59



In [ ]:
pred_label = []
true_label = []
for batch in tqdm(testloader):
    batch = [i.to(device) for i in batch]
    sent_id, masks, labels = batch
    preds = model(sent_id, masks)
    pred_label.extend(torch.argmax(preds, axis = 1))
    true_label.extend(labels)

  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
print(type(true_label))

<class 'list'>


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

true_label = torch.tensor(true_label)
pred_label = torch.tensor(pred_label)

confusion_matrix(true_label, pred_label)
print(classification_report(true_label, pred_label))